# Assignment: Data Collection and ETL

## Part 1: Get Data
"Pick a US state and (write a script to) download the most recent ACS data (at the block group level) for every block group in that state."

In [1]:
# set up

import requests
import json

In [2]:
# define parameters
# variables: https://api.census.gov/data/2019/acs/acs5/variables.html

year = '2019'
est_yr = '5' # 5-year estimates

state = '42' # 42 = pennsylvania

variables = {'B01001_001E':'pop_total',
             'B15003_001E':'pop_25plus',
             'B15003_017E':'pop_25plus_hsdiploma',
             'B15003_022E':'pop_25plus_bachelorsdeg',
             'B17017_001E':'num_hh',
             'B17017_002E':'num_hh_belowpovertylvl',
             'B28002_004E':'num_hh_withbroadband',}

In [3]:
# build url
# API guidance: https://www.census.gov/programs-surveys/acs/guidance/handbooks/api.html

variables_keys = ','.join(variables.keys()) # create variable key string

url = (f'''\
https://api.census.gov/data/{year}/acs/acs{est_yr}?\
get={variables_keys}&for=block+group:*&in=state:{state}+county:*\
''')

url

'https://api.census.gov/data/2019/acs/acs5?get=B01001_001E,B15003_001E,B15003_017E,B15003_022E,B17017_001E,B17017_002E,B28002_004E&for=block+group:*&in=state:42+county:*'

In [4]:
# request and format data

headers = {'Content-Type': 'application/json'}

response = requests.get(url, headers = headers)
if response.status_code == 200:
    data = json.loads(response.content.decode('utf-8'))

## Part 2: Transform/Prep
"Once you've downloaded the data, get it ready (with python code) so that it can be loaded into a postgres database table and is in a usable format for downstream applications."

In [5]:
# set up

import pandas as pd

In [6]:
# store as dataframe, apply headers

df = pd.DataFrame(data)
df = df.rename(columns = df.iloc[0]).drop([0])

In [7]:
# rename variables using variable dictionary parameter

df = df.rename(columns = variables).rename(columns = {'block group':'block_group'})

In [8]:
# rearrange columns - state, county, tract, block group first

df_cols = df.columns.to_list()
df_cols = df_cols[-4:] + df_cols[:-4]

df = df[df_cols]

In [9]:
# check data types

df.dtypes

state                      object
county                     object
tract                      object
block_group                object
pop_total                  object
pop_25plus                 object
pop_25plus_hsdiploma       object
pop_25plus_bachelorsdeg    object
num_hh                     object
num_hh_belowpovertylvl     object
num_hh_withbroadband       object
dtype: object

In [10]:
# modify data types (all integers except for geographic fields)

df[df.columns[4:]] = df[df.columns[4:]].apply(pd.to_numeric)

In [11]:
# describe, sanity check fields

df.describe(include = 'all')

,state,county,tract,block_group,pop_total,pop_25plus,pop_25plus_hsdiploma,pop_25plus_bachelorsdeg,num_hh,num_hh_belowpovertylvl,num_hh_withbroadband
count,9740,9740,9740,9740,9740.000000,9740.000000,9740.000000,9740.000000,9740.000000,9740.000000,9740.000000
unique,1,67,2326,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,42,101,010400,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,9740,1336,45,3217,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,1313.298768,919.325667,283.066427,174.678131,518.799384,62.859035,422.865092
std,NaN,NaN,NaN,NaN,687.389307,487.205586,166.959973,165.492069,258.754674,63.927078,235.432773
min,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,844.000000,587.000000,164.000000,65.000000,341.000000,20.000000,264.000000
50%,NaN,NaN,NaN,NaN,1161.000000,812.000000,258.000000,126.000000,463.000000,44.000000,368.000000
75%,NaN,NaN,NaN,NaN,1601.000000,1122.000000,375.000000,228.000000,635.000000,84.000000,522.000000


In [12]:
# view data

df.head()

,state,county,tract,block_group,pop_total,pop_25plus,pop_25plus_hsdiploma,pop_25plus_bachelorsdeg,num_hh,num_hh_belowpovertylvl,num_hh_withbroadband
1,42,129,801300,3,1779,1206,392,181,730,114,626
2,42,129,801900,3,2162,1521,749,256,797,48,600
3,42,129,801900,2,958,737,296,153,369,12,350
4,42,129,801900,1,1387,944,500,47,619,145,455
5,42,129,801900,4,2001,1689,617,214,1103,210,867


## Part 3: Load
"the data you've downloaded into a postgres database table. "

In [13]:
# set up

from sqlalchemy import create_engine
import psycopg2
import ohio.ext.pandas
import os

In [14]:
# get connection parameters from environment variables

db_connection = "postgresql://{}:{}@{}:{}/{}".format(
    os.getenv('acs_db_user'), 
    os.getenv('acs_db_password'), 
    os.getenv('acs_db_host'), 
    os.getenv('acs_db_port'), 
    os.getenv('acs_db_dbname'))

In [15]:
# create table query

create_table = ('''
DROP TABLE IF EXISTS acs.jandre_acs_data;

CREATE TABLE IF NOT EXISTS acs.jandre_acs_data (
    state varchar(10),
    county varchar(10),
    tract varchar(10),
    block_group varchar(10),
    pop_total int,
    pop_25plus int,
    pop_25plus_hsdiploma int,
    pop_25plus_bachelorsdeg int,
    num_hh int,
    num_hh_belowpovertylvl int,
    num_hh_withbroadband int);
''')

In [16]:
# connect to db, create table, insert data

engine = create_engine(db_connection)

with engine.connect() as connection:
    connection.execute(create_table)
    df.pg_copy_to('jandre_acs_data', connection, schema = 'acs', if_exists = 'replace', index = False)

In [17]:
# test

with engine.connect() as connection:
    result = connection.execute("SELECT * FROM acs.jandre_acs_data LIMIT 5")
    print(result.keys())
    for row in result:
        print(row)

['state', 'county', 'tract', 'block_group', 'pop_total', 'pop_25plus', 'pop_25plus_hsdiploma', 'pop_25plus_bachelorsdeg', 'num_hh', 'num_hh_belowpovertylvl', 'num_hh_withbroadband']
('42', '129', '801300', '3', 1779, 1206, 392, 181, 730, 114, 626)
('42', '129', '801900', '3', 2162, 1521, 749, 256, 797, 48, 600)
('42', '129', '801900', '2', 958, 737, 296, 153, 369, 12, 350)
('42', '129', '801900', '1', 1387, 944, 500, 47, 619, 145, 455)
('42', '129', '801900', '4', 2001, 1689, 617, 214, 1103, 210, 867)
